In [1]:
import graphlab
song_data = graphlab.SFrame('song_data.gl/')

[INFO] 1452690973 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/hzdy1994/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1452690973 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to gracedongy@gmail.com and will expire on December 27, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1254 - Server binary: /Users/hzdy1994/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452690973.log
[INFO] GraphLab Server Version: 1.7.1


In [4]:
artists = ['Kanye West', 'Foo Fighters', 'Taylor Swift', 'Lady GaGa']
for artist in artists:
    print artist, len(song_data[song_data['artist'] == artist]['user_id'].unique())

Kanye West 2522
Foo Fighters 2055
Taylor Swift 3246
Lady GaGa 2928


In [25]:
artist_count = song_data.groupby(key_columns='artist', 
                                 operations={'total_count': graphlab.aggregate.SUM('listen_count')})
artists2 = ['Taylor Swift', 'Kings Of Leon', 'Coldplay', 'Lady GaGa']
for artist2 in artists2:
    print artist2, artist_count[artist_count['artist'] == artist2]['total_count']

Taylor Swift [19376, ... ]
Kings Of Leon [43218, ... ]
Coldplay [35362, ... ]
Lady GaGa [12224, ... ]


In [27]:
print artist_count.sort('total_count')
print artist_count.sort('total_count', ascending=False)

+-------------------------------+-------------+
|             artist            | total_count |
+-------------------------------+-------------+
|        William Tabbert        |      14     |
|         Reel Feelings         |      24     |
| Beyoncé feat. Bun B and Sl... |      26     |
|             Diplo             |      30     |
|         Boggle Karaoke        |      30     |
|         harvey summers        |      31     |
|             Nâdiya            |      36     |
| Kanye West / Talib Kweli /... |      38     |
|        Aneta Langerova        |      38     |
|          Jody Bernal          |      38     |
+-------------------------------+-------------+
[3375 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
+------------------------+-------------+
|         artist         | total_count |
+------------------------+-------------+
|     Kings Of Leon      |    43218    |
|     Dwight

In [32]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [34]:
personalized_model = graphlab.item_similarity_recommender.create(train_data, 
                                                                        user_id='user_id', item_id='song')

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Warning: Ignoring columns song_id, listen_count, title, artist;
PROGRESS:     To use one of these as a target column, set target = <column_name>
PROGRESS:     and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 893580 observations with 66085 users and 9952 items.
PROGRESS:     Data prepared in: 1.52772s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 9952 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 1.03703         |
PROGRESS: | 2000            | 1.13226         |
PROGRESS: | 3000            | 1.22542         |
PROGRESS: | 4000            | 1.31837         |
PROGRESS: | 5000            | 1.40538         |
PROGRESS: | 6000            | 1.48967         |
PROGRESS: | 7000            | 1.57182         |


In [37]:
subset_test_users = test_data['user_id'].unique()[0:10000]
recommended_songs = personalized_model.recommend(subset_test_users,k=1)
print recommended_songs.head()

PROGRESS: recommendations finished on 1000/10000 queries. users per second: 1012.25
PROGRESS: recommendations finished on 2000/10000 queries. users per second: 1042.6
PROGRESS: recommendations finished on 3000/10000 queries. users per second: 1044.86
PROGRESS: recommendations finished on 4000/10000 queries. users per second: 1038.88
PROGRESS: recommendations finished on 5000/10000 queries. users per second: 1037.62
PROGRESS: recommendations finished on 6000/10000 queries. users per second: 1044.6
PROGRESS: recommendations finished on 7000/10000 queries. users per second: 1048.82
PROGRESS: recommendations finished on 8000/10000 queries. users per second: 1051.82
PROGRESS: recommendations finished on 9000/10000 queries. users per second: 1046.68
PROGRESS: recommendations finished on 10000/10000 queries. users per second: 1043.31
+-------------------------------+-------------------------------+
|            user_id            |              song             |
+----------------------------

In [38]:
artist_count = recommended_songs.groupby(key_columns='song', 
                                         operations={'count': graphlab.aggregate.COUNT()})
print artist_count.sort('count', ascending=False)

+--------------------------------+-------+
|              song              | count |
+--------------------------------+-------+
|          Undo - Björk          |  431  |
|     Secrets - OneRepublic      |  383  |
|    Revelry - Kings Of Leon     |  232  |
| You're The One - Dwight Yoakam |  169  |
| Fireflies - Charttraxx Karaoke |  123  |
|    Hey_ Soul Sister - Train    |  105  |
| Horn Concerto No. 4 in E f...  |   98  |
|    Sehr kosmisch - Harmonia    |   74  |
| OMG - Usher featuring will...  |   58  |
| Dog Days Are Over (Radio E...  |   54  |
+--------------------------------+-------+
[3135 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
